# Demonstration of use for deconvolving PAX data

In [ ]:
from pax_deconvolve.pax_simulations import simulate_pax

from pax_deconvolve import visualize


In [4]:
# Simulate some PAX data

# perform simulations with 10^7 average number of detected electrons (over entire dataset):
log10_num_electrons = 7.0
rixs_model = 'schlappa'   # use RIXS model chosen to approximate that in Schlappa's paper
photoemission_model = 'ag'   # 
data = simulate_pax.simulate_from_presets(7.0, 'schlappa', )

NameError: name 'simulate_pax' is not defined

In [3]:
# Assess convergence of PAX data

In [ ]:
# 